In [2]:
import pandas as pd
import numpy as np
import pickle

# Load your dataset
df = pd.read_csv("D:/Sang/ISONYLossesDeducible.csv", dtype=str)

def convert_to_int(x):
    try:
        return int(x)
    except:
        return None

def process_peril(df, peril):
    # Convert numeric columns
    numeric_columns = ['calc_claim', 'calc_paid_claim', peril, 'Deductible', 'CovA']
    for col in numeric_columns:
        df[col] = df[col].apply(convert_to_int)

    # Filter out data for the specific peril
    df_peril = df[df[peril] > 0]

    print(f'Data shape for {peril}: {df_peril.shape}')

    # Rename 'peril' column to 'net_loss'
    df_peril['net_loss'] = df_peril[peril]

    # Create cut points and labels
    cut_points = [-5000, 0, 225000, 250000, 275000, 300000, 325000, 350000, 375000, 400000, 425000, 450000, 475000, 500000, 550000, 600000, 650000, 700000, 750000, 800000, 900000, 1000000, float('inf')]
    labels = ['Unknown', '0-225K', '225-2.50K', '250-275K', '275-300K', '300-325K', '325-350K', '350-375K', '375-400K', '400-425K', '425-450K', '450-475K', '475-500K', '500-550K', '550-600K', '600-650K', '650-700K', '700-750K', '750-800K', '800-900K', '900-1000K', '1000K+']

    # Convert 'CovA' column to numeric
    df_peril['CovA_num'] = pd.to_numeric(df_peril['CovA'], errors='coerce')

    # Create 'CovA_cat' column with categories
    df_peril['CovA_cat'] = pd.cut(df_peril['CovA_num'], bins=cut_points, labels=labels, right=True)

    # Display the processed data
    display(df_peril[['CovA', 'CovA_num', 'CovA_cat', 'net_loss']])

    return df_peril

# List of perils
perils_input = input("Enter a comma-separated list of perils to process (e.g., Water2, Fire2): ")
perils = [peril.strip() for peril in perils_input.split(",")]

# Process data for each peril
for peril in perils:
    df_peril = process_peril(df, peril)

    # Define the list of deductibles
    deductibles = [100, 200, 250, 500, 750, 1000, 1500, 2000, 2500, 3000, 4000, 5000, 7500, 10000, 25000]

    # Initialize a dictionary to store results for each deductible
    results = {}

    # Loop through each deductible as the higher deductible
    for higher_deductible in deductibles:
        # Define a function to calculate net loss
        def calculate_net_loss(row):
            deductible = row['Deductible']
            if deductible < higher_deductible:
                return max(row['net_loss'] - row['calc_claim'] * (higher_deductible - deductible), 0)
            else:
                return 0

        # Apply the function to each row in the current peril DataFrame
        df_peril['netloss_to_higher_deductible'] = df_peril.apply(calculate_net_loss, axis=1)

        # Group by 'CovA_cat' and aggregate the sum of 'netloss' and 'net_loss' for each deductible
        netloss_to_higher_deductible = df_peril[df_peril['Deductible'] < higher_deductible].groupby('CovA_cat').agg({'netloss_to_higher_deductible': 'sum', 'net_loss': 'sum'})

        # Calculate 'ler_to_higher' for the current deductible
        netloss_to_higher_deductible['ler_to_higher'] = 1 - netloss_to_higher_deductible['netloss_to_higher_deductible'] / netloss_to_higher_deductible['net_loss']

        # Store the results in the dictionary with labels
        results[f'ler_to_{higher_deductible}'] = netloss_to_higher_deductible['ler_to_higher']
        results[f'netloss_to_{higher_deductible}'] = netloss_to_higher_deductible['netloss_to_higher_deductible']
        results[f'net_loss_to_{higher_deductible}'] = netloss_to_higher_deductible['net_loss']

    # Create a DataFrame to display the results for each deductible
    result_df = pd.DataFrame(results)
    print('result_df')
    display(result_df)
    
    def format_dollars(x):
        x=x*100
        return f'${x:.0f}'
    
    def format_percentage(x):
        x=x*100
        return f'%{x:.2f}'
    
    #Define the list of columns to have correct format
    dollar_columns = [col for col in result_df.columns if 'net_loss' in col or 'netloss' in col]
    percentage_columns=[col for col in result_df.columns if 'ler' in col]
    
    result_df=result_df.fillna(0)

    result_df[dollar_columns] = result_df[dollar_columns].applymap(format_dollars)
    result_df[percentage_columns]=result_df[percentage_columns].applymap(format_percentage)
    # Transpose the DataFrame
    result_df_T = result_df.T
    
    #dropping the first three records that are $100 deductible to $100 deductible.
    result_df_T = result_df_T.iloc[3:]

    # Save results to CSV and pickle files with the peril name
    result_df_T.to_csv(f'dynamic_{peril}_elr_all_deductible.csv')
    with open(f'dynamic_{peril}_elr_all_deductible.pickle', 'wb') as f:
        pickle.dump(result_df_T, f)

Data shape for Water2: (32270, 60)


C:\Users\shua\AppData\Local\Temp\3\ipykernel_34476\3606431031.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_peril['net_loss'] = df_peril[peril]
C:\Users\shua\AppData\Local\Temp\3\ipykernel_34476\3606431031.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_peril['CovA_num'] = pd.to_numeric(df_peril['CovA'], errors='coerce')
C:\Users\shua\AppData\Local\Temp\3\ipykernel_34476\3606431031.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,CovA,CovA_num,CovA_cat,net_loss
2,280000.0,280000.0,275-300K,6921
3,400000.0,400000.0,375-400K,2012
4,500000.0,500000.0,475-500K,11414
7,520000.0,520000.0,500-550K,14984
11,570000.0,570000.0,550-600K,21439
...,...,...,...,...
88417,260000.0,260000.0,250-275K,71068
88429,9999000.0,9999000.0,1000K+,17613
88430,410000.0,410000.0,400-425K,39824
88433,430000.0,430000.0,425-450K,577


C:\Users\shua\AppData\Local\Temp\3\ipykernel_34476\3606431031.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_peril['netloss_to_higher_deductible'] = df_peril.apply(calculate_net_loss, axis=1)
C:\Users\shua\AppData\Local\Temp\3\ipykernel_34476\3606431031.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_peril['netloss_to_higher_deductible'] = df_peril.apply(calculate_net_loss, axis=1)
C:\Users\shua\AppData\Local\Temp\3\ipykernel_34476\3606431031.py:68: SettingWithCopyWarning: 
A value is tryin

result_df


C:\Users\shua\AppData\Local\Temp\3\ipykernel_34476\3606431031.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_peril['netloss_to_higher_deductible'] = df_peril.apply(calculate_net_loss, axis=1)


,ler_to_100,netloss_to_100,net_loss_to_100,ler_to_200,netloss_to_200,net_loss_to_200,ler_to_250,netloss_to_250,net_loss_to_250,ler_to_500,...,net_loss_to_5000,ler_to_7500,netloss_to_7500,net_loss_to_7500,ler_to_10000,netloss_to_10000,net_loss_to_10000,ler_to_25000,netloss_to_25000,net_loss_to_25000
CovA_cat,,,,,,,,,,,,,,,,,,,,,
Unknown,NaN,0,0,0.019684,114548,116848,0.029526,113398,116848,0.045380,...,43515860,0.463025,24075473,44835349,0.547703,20278908,44835349,0.780094,9859572,44835349
0-225K,NaN,0,0,0.019907,167391,170791,0.029826,165697,170791,0.055588,...,23061391,0.572547,9875360,23102784,0.651037,8062010,23102784,0.835158,3823140,23192727
225-2.50K,NaN,0,0,0.021474,54682,55882,0.032211,54082,55882,0.050810,...,14188837,0.551495,6398135,14265460,0.629120,5290775,14265460,0.825854,2484274,14265460
250-275K,NaN,0,0,0.030397,28708,29608,0.045596,28258,29608,0.053646,...,10051428,0.559467,4464895,10135203,0.642049,3627911,10135203,0.821573,1808391,10135203
275-300K,NaN,0,0,0.014118,20950,21250,0.021176,20800,21250,0.039797,...,17053837,0.506779,8423999,17079574,0.583866,7107383,17079574,0.778639,3780784,17079748
300-325K,NaN,0,0,0.055866,3380,3580,0.083799,3280,3580,0.050276,...,11824530,0.505237,5857184,11838365,0.591805,4832365,11838365,0.809598,2254053,11838365
325-350K,NaN,0,0,0.033179,11656,12056,0.049768,11456,12056,0.031645,...,17896623,0.462322,9734762,18105192,0.538549,8354658,18105192,0.735850,4782483,18105192
350-375K,NaN,0,0,0.028615,10184,10484,0.042923,10034,10484,0.055660,...,12360051,0.459233,6712686,12413272,0.537941,5735668,12413272,0.742199,3206959,12439658
375-400K,NaN,0,0,0.030360,15969,16469,0.045297,15723,16469,0.052116,...,16594681,0.448209,9227243,16722362,0.528167,7890164,16722362,0.743799,4284281,16722362
